# Dashboard

In [1]:
from jupyter_dash import JupyterDash

In [42]:
import dash
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
import seaborn as sns

In [5]:
JupyterDash.infer_jupyter_proxy_config()

In [13]:
#Read in the ccombined data
finnish = pd.read_csv("finnish_data_combined.csv")
irish = pd.read_csv("irish_data_combined.csv")
irish.pop("Unnamed: 0")
finnish.pop("Unnamed: 0")

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
Name: Unnamed: 0, dtype: int64

In [14]:
finnish.head()

,Year,Total,Labour,Materials,Services,Residential buildings completion (No.)
0,1995,103.5,103.0,105.3,99.1,24526
1,1996,102.7,103.5,104.3,96.3,21372
2,1997,105.2,106.9,107.2,96.8,26955
3,1998,107.6,109.4,109.8,98.5,30529
4,1999,109.1,113.5,110.2,99.0,30537


In [15]:
irish.head()

,Quarter,House Construction Cost Index,No. of Units,Year
0,q1_2018,212.513233,14816.0,2018
1,q1_2019,214.525667,20324.0,2019
2,q1_2021,218.490518,34511.0,2021
3,q2_2018,213.003957,16351.0,2018
4,q2_2019,215.117876,22139.0,2019


In [6]:
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()

In [12]:
df.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN


In [33]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Irish'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Finnish'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=finnish['Year'].min(),
        max=finnish['Year'].max(),
        value=finnish['Year'].max(),
        marks={str(year): str(year) for year in finnish['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

In [34]:
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = finnish[finnish['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Residential buildings completion (No.)'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Residential buildings completion (No.)'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Residential buildings completion (No.)'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Residential buildings completion (No.)'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

In [35]:
def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

In [36]:
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = finnish[finnish['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [1]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for your data.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2023 15:12:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1684156255.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1684156255.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1684156255.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_3m1684156255.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [21/May/2023 15:12:43] "GET /_dash-component-suites/dash/dcc/as

## References

https://realpython.com/python-dash/

https://github.com/plotly/jupyter-dash/blob/master/notebooks/getting_started.ipynb